In [51]:
#!pip3 install nltk

In [52]:
import pandas as pd
import numpy as np
#from Get_Metadata import Get_Metadata, plot_confusion_matrix


from keras.preprocessing import sequence
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report
from keras import callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.layers import Embedding, LSTM, Dense
from keras.layers import Add, Input, Reshape,Concatenate, Conv1D, MaxPooling1D
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers


from string import punctuation

In [53]:
#@title Read input
def read_input():
  input_file = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskB_data_all.csv"
  ans_file = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskB_answers_all.csv"
  df_ans = pd.read_csv(ans_file, header=None, names=["id", "ans"])
  df_input = pd.read_csv(input_file)
  return df_input, df_ans

df_input, df_ans = read_input()

In [54]:
def stemLine(sentence):
    stemmer = nltk.PorterStemmer()
    translator = ''.join(ch for ch in sentence if ch not in punctuation)
   # translator=sentence.translate(string.maketrans('','', string.punctuation))
    translator = translator.lower()
    tokeniz = WordPunctTokenizer()
    tokens = tokeniz.tokenize(translator)
    #tokens = word_tokenize(translator)
    final = [stemmer.stem(tagged_word) for tagged_word in tokens]
    return " ".join(final)


df_input['FalseSentStemmed'] = df_input['FalseSent'].apply(lambda row: stemLine(row))
df_input['OptionAStemmed'] = df_input['OptionA'].apply(lambda row: stemLine(row))
df_input['OptionBStemmed'] = df_input['OptionA'].apply(lambda row: stemLine(row))
df_input['OptionCStemmed'] = df_input['OptionA'].apply(lambda row: stemLine(row))

In [55]:
df_input.head()

,id,FalseSent,OptionA,OptionB,OptionC,FalseSentStemmed,OptionAStemmed,OptionBStemmed,OptionCStemmed
0,0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,he pour orang juic on hi cereal,orang juic is usual bright orang,orang juic is usual bright orang,orang juic is usual bright orang
1,1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,he drink appl,appl juic are veri tasti and milk too,appl juic are veri tasti and milk too,appl juic are veri tasti and milk too
2,2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",jeff ran 100000 mile today,100000 mile is way to long for one person to b...,100000 mile is way to long for one person to b...,100000 mile is way to long for one person to b...
3,3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,i sting a mosquito,a human is a mammal,a human is a mammal,a human is a mammal
4,4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,a giraff is a person,giraff can drink water from a lake,giraff can drink water from a lake,giraff can drink water from a lake


In [56]:
le = LabelEncoder()
df_ans.columns = ['id', 'label']
df_ans['encoded'] = le.fit_transform(df_ans['label'])
df_ans.head()

,id,label,encoded
0,0,B,1
1,1,B,1
2,2,A,0
3,3,C,2
4,4,B,1


In [57]:
meta_data = df_input.drop(['FalseSent','OptionA', 'OptionB', 'OptionC'],axis=1)   #.iloc[:,:-1]
#meta_data = Get_Metadata(meta_data)
meta_data.head()

,id,FalseSentStemmed,OptionAStemmed,OptionBStemmed,OptionCStemmed
0,0,he pour orang juic on hi cereal,orang juic is usual bright orang,orang juic is usual bright orang,orang juic is usual bright orang
1,1,he drink appl,appl juic are veri tasti and milk too,appl juic are veri tasti and milk too,appl juic are veri tasti and milk too
2,2,jeff ran 100000 mile today,100000 mile is way to long for one person to b...,100000 mile is way to long for one person to b...,100000 mile is way to long for one person to b...
3,3,i sting a mosquito,a human is a mammal,a human is a mammal,a human is a mammal
4,4,a giraff is a person,giraff can drink water from a lake,giraff can drink water from a lake,giraff can drink water from a lake


In [58]:
from collections import Counter
all_text2_FalseSent = ' '.join(meta_data['FalseSentStemmed'])
# create a list of words
words = all_text2_FalseSent.split()
# Count all the words using Counter Method
count_words = Counter(words)
total_words = len(words)
sorted_words_FalseSent = count_words.most_common(total_words)

all_text2_OptA = ' '.join(meta_data['OptionAStemmed'])
# create a list of words
words = all_text2_OptA.split()
# Count all the words using Counter Method
count_words = Counter(words)
total_words = len(words)
sorted_words_OptA = count_words.most_common(total_words)

all_text2_OptB = ' '.join(meta_data['OptionBStemmed'])
# create a list of words
words = all_text2_OptB.split()
# Count all the words using Counter Method
count_words = Counter(words)
total_words = len(words)
sorted_words_OptB = count_words.most_common(total_words)

all_text2_OptC = ' '.join(meta_data['OptionCStemmed'])
# create a list of words
words = all_text2_OptC.split()
# Count all the words using Counter Method
count_words = Counter(words)
total_words = len(words)
sorted_words_OptC = count_words.most_common(total_words)

'''
sorted_words_FalseSent = get_sorted_words(meta_data['FalseSentStemmed'])
sorted_words_OptA = get_sorted_words(meta_data['OptionAStemmed'])
sorted_words_OptB = get_sorted_words(meta_data['OptionBStemmed'])
sorted_words_OptC = get_sorted_words(meta_data['OptionCStemmed'])
'''


"\nsorted_words_FalseSent = get_sorted_words(meta_data['FalseSentStemmed'])\nsorted_words_OptA = get_sorted_words(meta_data['OptionAStemmed'])\nsorted_words_OptB = get_sorted_words(meta_data['OptionBStemmed'])\nsorted_words_OptC = get_sorted_words(meta_data['OptionCStemmed'])\n"

In [59]:
vocab_to_int_FalseSent = {w:i+1 for i, (w,c) in enumerate(sorted_words_FalseSent)}
vocab_to_int_OptA = {w:i+1 for i, (w,c) in enumerate(sorted_words_OptA)}
vocab_to_int_OptB = {w:i+1 for i, (w,c) in enumerate(sorted_words_OptB)}
vocab_to_int_OptC = {w:i+1 for i, (w,c) in enumerate(sorted_words_OptC)}
print(len(vocab_to_int_OptA))
print(len(vocab_to_int_OptB))
print(len(vocab_to_int_OptC))

5203
5203
5203


In [60]:
FalseSent_int = []
for d in meta_data['FalseSentStemmed']:
    r = [vocab_to_int_FalseSent[w] for w in d.split()]
    FalseSent_int.append(r)

OptA_int = []
for d in meta_data['OptionAStemmed']:
    r = [vocab_to_int_OptA[w] for w in d.split()]
    OptA_int.append(r)
    
OptB_int = []
for d in meta_data['OptionBStemmed']:
    r = [vocab_to_int_OptB[w] for w in d.split()]
    OptB_int.append(r)
    
OptC_int = []
for d in meta_data['OptionCStemmed']:
    r = [vocab_to_int_OptC[w] for w in d.split()]
    OptC_int.append(r)

In [61]:
desc_len_FalseSent = [len(x) for x in FalseSent_int]
desc_len_OptA = [len(x) for x in OptA_int]
desc_len_OptB = [len(x) for x in OptB_int]
desc_len_OptC = [len(x) for x in OptC_int]

In [62]:
from keras.preprocessing.sequence import pad_sequences
pad_len_FalseSent = pd.Series(desc_len_FalseSent).max()
pad_len_OptA = pd.Series(desc_len_OptA).max()
pad_len_OptB = pd.Series(desc_len_OptB).max()
pad_len_OptC = pd.Series(desc_len_OptC).max()
pad_len = max([pad_len_FalseSent, pad_len_OptA, pad_len_OptB, pad_len_OptC])
#add padding to input
padded_FalseSent = pad_sequences(FalseSent_int, pad_len)
padded_OptA = pad_sequences(OptA_int, pad_len)
padded_OptB = pad_sequences(OptB_int, pad_len)
padded_OptC = pad_sequences(OptC_int, pad_len)
print(padded_FalseSent[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    6  378  801  634    8
   13 1488]


In [63]:
#FalseSent padding

data_pad_FalseSent = padded_FalseSent
train_rows = int(len(data_pad_FalseSent)*0.8)
test_rows = len(data_pad_FalseSent)-train_rows
#y_train=data['bot'][:train_rows]
X_train_0=data_pad_FalseSent[:train_rows]

X_test_0= data_pad_FalseSent[train_rows:]

print(X_train_0.shape, X_test_0.shape)

print(data_pad_FalseSent[:train_rows])
print(data_pad_FalseSent[:test_rows])
print(data_pad_FalseSent[train_rows:])

(8000, 30) (2000, 30)
[[   0    0    0 ...    8   13 1488]
 [   0    0    0 ...    6   54  132]
 [   0    0    0 ... 3103 1065  276]
 ...
 [   0    0    0 ...  191    3  701]
 [   0    0    0 ...   10  480   54]
 [   0    0    0 ...  292   33 1270]]
[[   0    0    0 ...    8   13 1488]
 [   0    0    0 ...    6   54  132]
 [   0    0    0 ... 3103 1065  276]
 ...
 [   0    0    0 ...   39    3   24]
 [   0    0    0 ...    3    1  163]
 [   0    0    0 ...  193   20  142]]
[[   0    0    0 ...  411    9 4650]
 [   0    0    0 ...    4  694  368]
 [   0    0    0 ...   12  149  190]
 ...
 [   0    0    0 ...   13  183 1007]
 [   0    0    0 ...    8    1  291]
 [   0    0    0 ...    8   15  501]]


In [64]:
#OptionA padding

data_pad_OptA = padded_OptA
train_rows = int(len(data_pad_OptA)*0.8)
test_rows = len(data_pad_OptA)-train_rows
#y_train=data['bot'][:train_rows]
X_train_1=data_pad_OptA[:train_rows]

X_test_1= data_pad_OptA[train_rows:]

print(X_train_1.shape, X_test_1.shape)

(8000, 30) (2000, 30)


In [65]:
#OptionB padding

data_pad_OptB = padded_OptB
train_rows = int(len(data_pad_OptB)*0.8)
test_rows = len(data_pad_OptB)-train_rows
#y_train=data['bot'][:train_rows]
X_train_2=data_pad_OptB[:train_rows]

X_test_2= data_pad_OptB[train_rows:]

print(X_train_2.shape, X_test_2.shape)

(8000, 30) (2000, 30)


In [66]:
#OptionC padding

data_pad_OptC = padded_OptC
train_rows = int(len(data_pad_OptC)*0.8)
test_rows = len(data_pad_OptC)-train_rows
#y_train=data['bot'][:train_rows]
X_train_3=data_pad_OptC[:train_rows]

X_test_3= data_pad_OptC[train_rows:]

print(X_train_3.shape, X_test_3.shape)

(8000, 30) (2000, 30)


In [67]:
#one-hot encoding truth values 
from keras.utils import to_categorical

encoded_labels = df_ans['encoded']
print(len(df_ans))

y = encoded_labels
#y_svm_train = y[:train_rows]
#y_svm_test = y[:test_rows]
y = to_categorical(y)
y_train = y[:train_rows]
y_train = y_train
y_test = y[train_rows:]
print(np.array(encoded_labels).shape,y_train.shape)
# invert encoding
inverted = np.argmax(y_train[0])
if(inverted>0): print("true")

10000
(10000,) (8000, 3)
true


## Multi-class classification using LSTM model

In [68]:
# create the model

def task4A_LSTM_model(X_train,y_train):
    
    X_train_0,X_train_1,X_train_2,X_train_3 = X_train
    embedding_vecor_length = X_train_0.shape[1]
   # model0 = Sequential()
    batch_size=45
    #model0
    input0 = Input(shape=(X_train_0.shape[1],))
    x0 = Embedding(len(vocab_to_int_FalseSent)+1, embedding_vecor_length, input_length=X_train_0.shape[1])(input0)
    x0 = LSTM(20)(x0) #,batch_input_shape=(batch_size,19,1957)))
    
    #model1
   # model1 = Sequential()
    input1 = Input(shape=(X_train_1.shape[1],))
    x1 = Embedding(len(vocab_to_int_OptA)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input1)
    x1 = LSTM(20)(x1) #,batch_input_shape=(batch_size,19,1957)))
    
    #model2
    # model1 = Sequential()
    input2 = Input(shape=(X_train_2.shape[1],))
    x2 = Embedding(len(vocab_to_int_OptB)+1, embedding_vecor_length, input_length=X_train_2.shape[1])(input2)
    x2 = LSTM(20)(x2) #,batch_input_shape=(batch_size,19,1957)))
    
    #model3
    # model1 = Sequential()
    input3 = Input(shape=(X_train_3.shape[1],))
    x3 = Embedding(len(vocab_to_int_OptC)+1, embedding_vecor_length, input_length=X_train_3.shape[1])(input3)
    x3 = LSTM(20)(x3) #,batch_input_shape=(batch_size,19,1957)))
    
    model = Sequential()
    x = Concatenate()([x0,x1,x2,x3])
    print(x.shape)
    x = Reshape((4,20),)(x)
    x = LSTM(20, activation = 'relu')(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(3, activation="softmax")(x)
    model = Model(inputs=[input0, input1, input2, input3], outputs=x)
    #opt = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    es = callbacks.EarlyStopping(monitor='val_loss', mode='min', min_delta=0,patience=7)
    mc = callbacks.ModelCheckpoint('task4A_LSTM_model.h5')
    
    model.fit([X_train_0,X_train_1,X_train_2,X_train_3], y_train, epochs=30, shuffle=True, callbacks=[mc],validation_split=0.1)
    return model

#print(model.summary()
model= task4A_LSTM_model([X_train_0, X_train_1, X_train_2, X_train_3],y_train)

(?, 80)
Train on 7200 samples, validate on 800 samples
Epoch 1/30
7200/7200 [==============================] - 6s 866us/step - loss: 1.0408 - accuracy: 0.4101 - val_loss: 0.9428 - val_accuracy: 0.4963
Epoch 2/30
7200/7200 [==============================] - 6s 778us/step - loss: 0.8620 - accuracy: 0.5393 - val_loss: 0.9351 - val_accuracy: 0.5038
Epoch 3/30
7200/7200 [==============================] - 6s 844us/step - loss: 0.7349 - accuracy: 0.6004 - val_loss: 0.9989 - val_accuracy: 0.4762
Epoch 4/30
7200/7200 [==============================] - 6s 828us/step - loss: 0.6173 - accuracy: 0.6582 - val_loss: 1.2569 - val_accuracy: 0.4625
Epoch 5/30
7200/7200 [==============================] - 8s 1ms/step - loss: 0.5238 - accuracy: 0.7286 - val_loss: 1.4928 - val_accuracy: 0.4588
Epoch 6/30
7200/7200 [==============================] - 7s 1ms/step - loss: 0.4172 - accuracy: 0.8018 - val_loss: 2.0440 - val_accuracy: 0.4712
Epoch 7/30
7200/7200 [==============================] - 7s 964us/step - l

In [69]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 30)           0                                            
____________________________________________________________________________________________

In [70]:
# Final evaluation of the model
predictions = model.predict([X_test_0, X_test_1, X_test_2,X_test_3])
inverted = [np.argmax(i) for i in predictions]
truth= [np.argmax(i) for i in y_test]
#accuracy
scores = model.evaluate([X_test_0, X_test_1, X_test_2,X_test_3], y_test, verbose=1)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))
print(classification_report(truth, inverted))

2000/2000 [==============================] - 0s 171us/step
[7.864671447753906, 0.41749998927116394]
Accuracy: 41.75%
              precision    recall  f1-score   support

           0       0.52      0.46      0.49       646
           1       0.37      0.43      0.39       675
           2       0.39      0.37      0.38       679

    accuracy                           0.42      2000
   macro avg       0.43      0.42      0.42      2000
weighted avg       0.42      0.42      0.42      2000



In [71]:
def to_submission_form(y_test, y_pred, model_name):
    pred_labels = open("pred_labels.csv", "w")
    y_test_labels = le.inverse_transform(y_test)
    y_pred_labels = le.inverse_transform(y_pred)
    final_result = pd.DataFrame({"gold": y_test_labels, "pred": y_pred_labels})
    final_result.to_csv(model_name + "gold_labels.csv", columns = ["gold"], header=False)
    final_result.to_csv(model_name + "pred_labels.csv", columns = ["pred"], header=False)

In [72]:
to_submission_form(truth, inverted, "LSTM")

### Result for multi-class classification using LSTM model

In [73]:
!python3 taskB_scorer.py --gold-labels LSTMgold_labels.csv --pred-labels LSTMpred_labels.csv

Accuracy: 41.7500%


## CNN for multi-class classification

In [74]:
def create_conv_model(X_train, y_train):
    X_train_0,X_train_1,X_train_2,X_train_3 = X_train
    model_conv = Sequential()
    embedding_vecor_length = X_train_0.shape[1]
   # model0 = Sequential()
    batch_size=45
    #model0
    input0 = Input(shape=(X_train_0.shape[1],))
    x0 = Embedding(len(vocab_to_int_FalseSent)+1, embedding_vecor_length, input_length=X_train_0.shape[1])(input0)
    x0 = (Conv1D(25, 3, activation='relu',name='conv_0'))(x0)
    #x0 = MaxPooling1D(pool_size=3)(x0)
    
    #model1
    input1 = Input(shape=(X_train_1.shape[1],))
    x1 = Embedding(len(vocab_to_int_OptA)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input1)
    x1 = (Conv1D(25, 3, activation='relu',name='conv_1'))(x1)
    #x1 = MaxPooling1D(pool_size=3)(x1)
    
    #model2
    input2 = Input(shape=(X_train_2.shape[1],))
    x2 = Embedding(len(vocab_to_int_OptB)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input2)
    x2 = (Conv1D(25, 3, activation='relu',name='conv_2'))(x2)
    #x1 = MaxPooling1D(pool_size=3)(x1)
    
    #model3
    input3 = Input(shape=(X_train_3.shape[1],))
    x3 = Embedding(len(vocab_to_int_OptC)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input3)
    x3 = (Conv1D(25, 3, activation='relu',name='conv_3'))(x3)
    #x1 = MaxPooling1D(pool_size=3)(x1)
    
    x = Concatenate()([x0,x1,x2,x3])
    x = Conv1D(25, 3, activation='relu',name='conv')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = LSTM(10)(x)
    x = Dense(3, activation='sigmoid')(x)
    
    model_conv = Model(input = [input0,input1,input2,input3], output = x)
    model_conv.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    model_conv.summary()
    
    es = callbacks.EarlyStopping(monitor='loss', mode='min', min_delta=0,patience=1)
    mc = callbacks.ModelCheckpoint('task4A_CNN_model.h5')
    
    model_conv.fit([X_train_0,X_train_1,X_train_2,X_train_3], y_train, epochs=30,shuffle=False, callbacks=[mc, es],validation_split=0.2)
    return model_conv

model_conv = create_conv_model([X_train_0,X_train_1,X_train_2,X_train_3], y_train)

# Final evaluation of the model
predictions = model_conv.predict([X_test_0,X_test_1,X_test_2,X_test_3])
inverted = [np.argmax(i) for i in predictions]
truth= [np.argmax(i) for i in y_test]
#accuracy
scores = model_conv.evaluate([X_test_0,X_test_1], y_test, verbose=1)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))
print(classification_report(truth,inverted))


/usr/lib/python3/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 30)           0                                            
____________________________________________________________________________________________

6400/6400 [==============================] - 3s 535us/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.0104 - val_accuracy: 0.4375
Epoch 29/30
6400/6400 [==============================] - 3s 528us/step - loss: 9.6990e-04 - accuracy: 1.0000 - val_loss: 3.0504 - val_accuracy: 0.4375
Epoch 30/30
6400/6400 [==============================] - 4s 549us/step - loss: 8.6823e-04 - accuracy: 1.0000 - val_loss: 3.0901 - val_accuracy: 0.4356


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 2 arrays: [array([[   0,    0,    0, ...,  411,    9, 4650],
       [   0,    0,    0, ...,    4,  694,  368],
       [   0,    0,    0, ...,   12,  149,  190],
       ...,
       [   0,    0,    0, ...,   13, ...

In [ ]:
to_submission_form(truth, inverted, "CNN")

In [ ]:
!python3 taskB_scorer.py --gold-labels CNNgold_labels.csv --pred-labels CNNpred_labels.csv

## Capsule Networks for multi-class classification

In [ ]:
!git clone https://github.com/XifengGuo/CapsNet-Keras.git capsnet-keras

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard

cp_meta = ModelCheckpoint(filepath="CNN_capsnet_model_Semval_Task4A.h5",save_best_only=True,verbose=1)
tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=False)

In [ ]:
from keras.layers import Concatenate,Dropout,Reshape
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
from keras import layers, models, optimizers

def CapsNet_Meta(X_train, input_shape, n_class, routings):
    """
    A Capsule Network on text data.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
    `eval_model` can also be used for training.
    """
    X_train_0, X_train_1, X_train_2, X_train_3 = X_train
    model_capsnet = Sequential()
    embedding_vecor_length = X_train_0.shape[1]
    batch_size=45
    
    #model0
    input0 = Input(shape=(X_train_0.shape[1],))
    x0 = Embedding(len(vocab_to_int_FalseSent)+1, embedding_vecor_length, input_length=X_train_0.shape[1])(input0)
    x0 = (Conv1D(25, 3, activation='relu',name='conv_0'))(x0)
    #x0 = MaxPooling1D(pool_size=3)(x0)
    
    #model1
    input1 = Input(shape=(X_train_1.shape[1],))
    x1 = Embedding(len(vocab_to_int_OptA)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input1)
    x1 = (Conv1D(25, 3, activation='relu',name='conv_1'))(x1)
    
    #model2
    input2 = Input(shape=(X_train_2.shape[1],))
    x2 = Embedding(len(vocab_to_int_OptB)+1, embedding_vecor_length, input_length=X_train_2.shape[1])(input2)
    x2 = (Conv1D(25, 3, activation='relu',name='conv_1'))(x2)
    
    #model1
    input3 = Input(shape=(X_train_3.shape[1],))
    x3 = Embedding(len(vocab_to_int_OptC)+1, embedding_vecor_length, input_length=X_train_1.shape[1])(input3)
    x3 = (Conv1D(25, 3, activation='relu',name='conv_1'))(x3)
    
    x = Concatenate()([x0,x1,x2,x3])
    print(x.shape)
    x = Reshape((6,7,25),)(x)
    conv2 = layers.Conv2D(filters=25, kernel_size=3, strides=1, padding='valid', activation='relu', name='conv2')(x)
    print(conv2.shape)

    
    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv2, dim_capsule=4, n_channels=32, kernel_size=3, strides=1, padding='valid')

    print(primarycaps)
    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
    name='digitcaps')(primarycaps)
    print(digitcaps.shape)
    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    
    #print(out_caps.shape)
    
    #print(con_M.shape)
    out_caps = Length(name='capsnet')(digitcaps)
    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y]) # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps) # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(10, activation='relu', input_dim=16*n_class))
    decoder.add(layers.Dense(20, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model(inputs=[input0, input1, input2, input3], outputs=out_caps)
   # eval_model = models.Model([x,meta_d], out_caps)

    # manipulate model
   # noise = layers.Input(shape=(n_class, 16))
   # noised_digitcaps = layers.Add()([digitcaps, noise])
   # masked_noised_y = Mask()([noised_digitcaps, y])
   # manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model  #, eval_model  #, manipulate_model

model_capsnet = CapsNet_Meta([X_train_0, X_train_1, X_train_2, X_train_3],input_shape=X_train_0.shape[1:], n_class=3, routings=3)

model_capsnet.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])  #loss_weights=[1.],
model_capsnet.summary()

es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=1, min_lr=0.00001)

for i in range(1):
    history=model_capsnet.fit([X_train_0, X_train_1, X_train_2, X_train_3], y_train, batch_size=45, epochs=15,
              validation_split=0.1, callbacks=[cp_meta]).history
    model_capsnet.evaluate([X_test_0, X_test_1, X_test_2, X_test_3], y_test)
    predictions = model_capsnet.predict([X_test_0, X_test_1, X_test_2, X_test_3])
    print(predictions)
plt.plot(history['loss'])